## TESTE ANN

Import dependencies

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

#from sklearn.cross_validation import train_test_split

Define to get all features (id excluded) ...

In [2]:
def preprocess_x(df_x):
    return df_x[['x0','y0','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6','x7','y7','x8','y8','x9','y9']].values

Define to get all results (id excluded)...

In [3]:
def preprocess_y(df_y):
    return df_y[["slope", "intercept"]].values

Load train datasets and process collumns

In [4]:
x_data = pd.read_csv("train_100k.csv")
x_data = preprocess_x(x_data)

y_data = pd.read_csv("train_100k.truth.csv")
y_data =  preprocess_y(y_data)

Load test datasets and process collumns

In [5]:
x_data_test = pd.read_csv("train_100k.csv")
x_data_test = preprocess_x(x_data_test)

## The ANN
The first, I'm tring to figure out how transfor the two output in one point. So i read again the problem set and decide only to make a simple artificial neural network with two output in a linear activation function (no activation at all).

### ANN Configuration  
Some tests:  
Epoch: 1000 cost = 0.138191439 cost2 = 47.019507828 4x200  
Epoch: 1000 cost = 0.017634519 cost2 = 11.450007486 5x400  
Epoch: 1000 cost = 0.008747602 cost2 = 6.681912537 4x500 learning rate 0.0001  

So, how more hidden reurons and smaller learning rate, more performance in the results...

In [6]:
learning_rate = 0.0001
training_epochs = 5000
batch_size = 1000
display_step = 100
train_data_size = 100000

n_hidden = 500 # number of neurons on hidden layer

n_input = 20
n_output = 2

Create features and response tensors

In [7]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_output])

Create tensors w'll be optimized:  
5 hidden layers  
1 output layer  
5 hidden bias  
1 output bias  

In [8]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden],stddev=0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h5': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden, n_output],stddev=0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden])),
    'b2': tf.Variable(tf.random_normal([n_hidden])),
    'b3': tf.Variable(tf.random_normal([n_hidden])),
    'b4': tf.Variable(tf.random_normal([n_hidden])),
    'b5': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

Make a fully connected ANN ..

In [9]:
def make_network(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.tanh(layer_1)

    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.tanh(layer_2)

    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.tanh(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.tanh(layer_4)
    
    layer_5 = tf.add(tf.matmul(layer_4, weights['h5']), biases['b5'])
    layer_5 = tf.nn.tanh(layer_5)

    out_layer = tf.matmul(layer_5, weights['out']) + biases['out']
    return out_layer

Run feed forward ...

In [10]:
y_pred = make_network(x, weights, biases)

Adjust MSE

In [11]:
slope_cost = tf.reduce_mean(tf.square(y_pred[:,0]-y[:,0]))

Adjust MAE

In [12]:
intercept_cost = tf.reduce_mean(tf.abs(y_pred[:,1]-y[:,1]))

Run backward propagation

In [13]:
slope_optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(slope_cost)
intercept_optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(intercept_cost)

Create batches to train in more small steps an

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(train_data_size / batch_size)
    for epoch in range(training_epochs):
        avg_cost = 0
        avg_cost2 = 0
        
        for i in range(total_batch):
            batch_x, batch_y = x_data[i * batch_size:min(i * batch_size + batch_size, train_data_size), :], \
                             y_data[i * batch_size:min(i * batch_size + batch_size, train_data_size), :]
            _,  slope_c, _, intercept_c = sess.run([slope_optimizer,  slope_cost, intercept_optimizer,  intercept_cost], feed_dict={x: batch_x, y: batch_y})
            
            avg_cost += slope_c / total_batch
            avg_cost2 += intercept_c / total_batch
        
        if (epoch+1) % display_step == 0:
            print('Epoch:', (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'cost2 =', '{:.9f}'.format(avg_cost2))
    
    # Generate train result 
    r = sess.run(y_pred, feed_dict = {x: x_data})
    r = pd.DataFrame(r, columns=["slope", "intercept"])
    r.to_csv("submission.train_100k.csv", index=True, index_label='id')
    
    # Generate test result
    r = sess.run(y_pred, feed_dict = {x: x_data_test})
    r = pd.DataFrame(r, columns=["slope", "intercept"])
    r.to_csv("submission.test_100k.csv", index=True, index_label='id')

Epoch: 100 cost = 0.032156546 cost2 = 25.504869118
Epoch: 200 cost = 0.019468157 cost2 = 17.121879854
Epoch: 300 cost = 0.016755923 cost2 = 12.418868332
Epoch: 400 cost = 0.014511940 cost2 = 11.578304768
Epoch: 500 cost = 0.022932198 cost2 = 11.408332253
Epoch: 600 cost = 0.018589433 cost2 = 8.254057159
Epoch: 700 cost = 0.011675939 cost2 = 11.253377357
Epoch: 800 cost = 0.012578157 cost2 = 10.440568056
Epoch: 900 cost = 0.011732464 cost2 = 9.041368961
Epoch: 1000 cost = 0.013583385 cost2 = 7.999374356
Epoch: 1100 cost = 0.006592688 cost2 = 6.349081912
Epoch: 1200 cost = 0.007857357 cost2 = 6.502352700
Epoch: 1300 cost = 0.005654996 cost2 = 5.371522698
Epoch: 1400 cost = 0.004334838 cost2 = 5.226565807
Epoch: 1500 cost = 0.005200569 cost2 = 5.618376720
Epoch: 1600 cost = 0.006270270 cost2 = 7.081418505
Epoch: 1700 cost = 0.005161653 cost2 = 5.287339933
Epoch: 1800 cost = 0.004054777 cost2 = 4.678557041
Epoch: 1900 cost = 0.007042963 cost2 = 5.421184981
Epoch: 2000 cost = 0.010520406 co

cost = MSE  
cost2 = MAE

```bash
$ python evaluate.py train_100k.truth.csv submission.train_100k.csv  
Slope mse: 0.0114102844069  
Intercept mae: 2.29707436696  
```